In [1]:
%pylab inline --no-import-all
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
import os

folder = '../../../../crispy'
print(folder)
if folder not in sys.path: sys.path.append(folder)

from crispy.params import Params
folder = '../../../crispy'

from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.IFS import propagateIFS
from crispy.IFS import reduceIFSMap
from crispy.tools.postprocessing import process_SPC_IFS,SNR_spectrum
from crispy.tools.imgtools import scale2imgs,bowtie
from crispy.unitTests import testCreateFlatfield

#os.chdir(folder)
par = Params(folder)
par.saveRotatedInput = False
par.saveDetector = False


Populating the interactive namespace from numpy and matplotlib
../../../../crispy


In [2]:
par.savePoly=True
par.saveRotatedInput=True
testCreateFlatfield(par,useQE=True)


crispy - INFO - Reduced cube will have 44 wavelength bins
crispy - INFO - The number of input pixels per lenslet is 5.000000
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming endpoints wavelist is given


Process Consumer-5:
Process Consumer-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process Consumer-1:
    self.run()
Process Consumer-4:
  File "../../../../crispy/crispy/tools/par_utils.py", line 22, in run
Process Consumer-7:
    self.result_queue.put(next_task())
Process Consumer-3:
Traceback (most recent call last):
  File "../../../../crispy/crispy/tools/par_utils.py", line 31, in __call__
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
Process Consumer-2:
Traceback (most recent call last):
Process Consumer-8:
Traceback (most recent call last):
  File "/Users/mrizzo/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
    self.run()
  File "/Users/mrizzo/anaconda/lib/

KeyboardInterrupt: 

### reduce flatfield

In [ ]:
reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits')

In [ ]:
par.nonoise=True
os5folder = '/Users/mrizzo/IFS/OS5/with_lowfc'
offaxispsf= '/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
signal,noise,noiseleft = process_SPC_IFS(par,os5folder,offaxispsf,
                outdir_time_series = '/Users/mrizzo/IFS/OS5/',outdir_detector='/Users/mrizzo/IFS/OS5/OS5_detector',
                outdir_average='/Users/mrizzo/IFS/OS5/OS5_average',
                pp_fact=1.00,
                t_zodi=0.00,
                subtract_ref_psf=False,
                process_cubes=False,
                process_offaxis=False,
                process_detector=True,
                take_averages=True,
                parallel=False)
np.savetxt(par.exportDir+"/signal.txt",signal)
np.savetxt(par.exportDir+"/noise.txt",noise)
np.savetxt(par.exportDir+"/noiseleft.txt",noiseleft)

In [ ]:
from crispy.tools.reduction import calculateWaveList
signal = np.loadtxt(par.exportDir+"/signal.txt")
noise = np.loadtxt(par.exportDir+"/noise.txt")
lam_midpts,junk = calculateWaveList(par)
wavelist = 770*np.linspace(1.-0.18/2.,1.+0.18/2.,45)
SNR_spectrum(lam_midpts,signal,noise,lam_contrast=wavelist,outfolder=par.exportDir+"/",FWHM=2.*44./19.)



In [ ]:
from astropy.io import fits
test = fits.open('/local/data/nicolaus2/mrizzo/crispy/crispy/OS5/os5_spc_031_targetstar_IFS.fits')[1].data
print(np.sum(test))

In [ ]:
from crispy.tools.inputScene import adjust_krist_header
from crispy.tools.image import Image
offaxispsf= '/Users/mrizzo/IFS/OS5/offaxis/spc_offaxis_psf.fits'
cube = Image(offaxispsf)
cube.header
adjust_krist_header(cube,lamc=770.)
cube.header

## Test least-square normalization of cube

In [ ]:
from crispy.tools.image import Image
img2 = Image(par.exportDir+"/average_ref_star_detector_red_optext.fits")
img1 = Image(par.exportDir+"/average_target_star_detector_red_optext.fits")
ydim,xdim = img1.data[0].shape
mask,junk = bowtie(img1.data[0],ydim//2,xdim//2,openingAngle=65,
            clocking=-par.philens*180./np.pi,IWApix=4*0.77/0.6,OWApix=22*0.77/0.6,
            export=None,twomasks=False)
coefs,diff = scale2imgs(img1,img2,mask=mask,returndiff = True)
Image(data=diff).write(par.exportDir+"/test_lstsqdiff.fits")
print(coefs)

In [ ]:
from crispy.tools.rotate import Rotate,Shift
#test = Rotate(diff,par.philens,clip=False)
outcube = [Rotate(diff[i],-par.philens,True,order=1) for i in range(diff.shape[0])]
Image(data=np.array(outcube)).write(par.exportDir+"/test_rotate_cube.fits")
